In [1]:
import pandas as pd
import numpy as np
#from src.config import Config
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

In [2]:
raw_data1 = pd.read_csv("/Users/salihburakgurhan/Python/end_to_end_sales_estimator/data/Blackbox US Electronics.csv", delimiter=";")
raw_data2 = pd.read_csv("/Users/salihburakgurhan/Python/end_to_end_sales_estimator/data/Keepa US Electronics.csv")
merged_data = raw_data1.merge(raw_data2, on="ASIN")
merged_data.head()

,URL,ASIN,Title_x,Brand,Fulfillment,Category,BSR,Subcategory,Price,Price Trend (90 days) (%),...,Number of Images,Variation Count,Sales to Reviews,Buy Box 🚚: Current,Title_y,Sales Rank: Current,Sales Rank: 90 days avg.,Sales Rank: Drops last 90 days,Sales Rank: Subcategory Sales Ranks,Bought in past month
0,https://amazon.com/dp/B0933BVK6T,B0933BVK6T,Apple AirTag,Apple,Amazon,Electronics,1,Finders,27.99,-1.0,...,6,0,0.23,$ 27.99,Apple AirTag,1.0,3,33,# 1 | Top 1% | Item Finders,10000
1,https://amazon.com/dp/B0932QJ2JZ,B0932QJ2JZ,Apple AirTag 4 Pack,Apple,Amazon,Electronics,3,Finders,84.99,-2.0,...,6,0,0.36,$ 84.99,Apple AirTag 4 Pack,3.0,3,98,# 2 | Top 1% | Item Finders,10000
2,https://amazon.com/dp/B0CHWRXH8B,B0CHWRXH8B,Apple AirPods Pro (2nd Generation) Wireless Ea...,Apple,Amazon,Electronics,2,Earbud Headphones,189.00,-6.0,...,6,1,4.48,$ 189.00,Apple AirPods Pro (2nd Generation) Wireless Ea...,2.0,5,100,# 1 | Top 1% | Earbud & In-Ear Headphones,10000
3,https://amazon.com/dp/B07PXGQC1Q,B07PXGQC1Q,Apple AirPods (2nd Generation) Wireless Ear Bu...,Apple,Amazon,Electronics,4,Earbud Headphones,99.00,-1.0,...,6,0,0.06,$ 99.00,Apple AirPods (2nd Generation) Wireless Ear Bu...,4.0,5,105,# 2 | Top 1% | Earbud & In-Ear Headphones,10000
4,https://amazon.com/dp/B09BKHHL5H,B09BKHHL5H,Surge Protector USB Outlet Extender - 5-Outlet...,Lvetek,FBA,Electronics,11,Surge Protectors,12.98,-8.0,...,8,0,1.93,$ 12.98,Surge Protector USB Outlet Extender - 5-Outlet...,13.0,41,197,# 1 | Top 1% | Surge Protectors,10000


# Data Cleaning

In [3]:
data = merged_data[["BSR", "Monthly Sales"]].copy()

In [4]:
print(data.isna().sum().sum())
data.dropna(inplace=True)
print(data.isna().sum().sum())

0
0


In [5]:
print(data.duplicated().sum().sum())
data.drop_duplicates(inplace=True)
print(data.duplicated().sum().sum())

91
0


In [6]:
data.shape

(4093, 2)

# Data Preprocessing

In [7]:
X = data.drop("Monthly Sales", axis=1)
y = data["Monthly Sales"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

# Hyperparameter Selection

In [9]:
print(np.arange(10,210,50))

[ 10  60 110 160]


In [10]:
best_params = {
            "n_estimators" : 450,
            "criterion" : 'squared_error',
            "max_leaf_nodes" : 160,
            "random_state" : 42,
            "max_depth":5          
        }

model = RandomForestRegressor(**best_params)

#grid = GridSearchCV(estimator=model, param_grid=grid_params, cv=5)
#grid.fit(X_train, y_train)
#print(f"Score: {grid.best_score_}, Params: {grid.best_params_}")

In [12]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Model Evaluation Results")
print(f"R2 Score: {r2}")
print(f"MAPE: {mape}")
print(f"RMSE: {rmse}")

Model Evaluation Results
R2 Score: 0.8771537720981933
MAPE: 0.29594649503140186
RMSE: 883.6232327103189


In [14]:
model = RandomForestRegressor(**best_params).fit(X,y)

In [15]:
model.predict(np.array(12759).reshape(-1,1))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([315.73015873])

In [16]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'my_model.joblib')


['my_model.joblib']

In [ ]:
# Load the model from the file
model = joblib.load('my_model.joblib')

# Use the loaded model to make predictions
predictions = model.predict(X_test)
